# PostgreSQL Database Explorer

This notebook provides tools to explore the database in both local and RBC environments.
You can examine the structure and content of various database tables, particularly for different subagent sources.

## Features:
- Connect to either local or RBC database
- Explore catalog and content tables
- Examine specific document sources (internal_wiki, etc.)
- Compare document IDs and their formats

In [ ]:
# Import required modules
import sys
import os
import pandas as pd
from IPython.display import display, HTML
from datetime import datetime

# Add the project root to the path if needed
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import directly from the module
import iris.src.initial_setup.db_config as db_config
from iris.src.chat_model.model_settings import ENVIRONMENT

In [ ]:
# Function to connect to the database
def connect_to_db(env=None):
    """Connect to the database in the specified environment."""
    if env is None:
        # Use the default environment from model_settings
        env = ENVIRONMENT
    
    print(f"Connecting to database in '{env}' environment...")
    conn = db_config.connect_to_db(env)
    if conn:
        print("Connected successfully! 🎉\n")
        return conn
    else:
        print("Failed to connect to database.")
        return None

# Connect to default environment
conn = connect_to_db()

## Database Overview

Get a quick overview of the database tables and their record counts.

In [ ]:
def get_database_overview(conn):
    """Get an overview of the database tables."""
    if not conn:
        return
    
    with conn.cursor() as cur:
        # Check catalog records
        cur.execute("SELECT COUNT(*) FROM apg_catalog")
        catalog_count = cur.fetchone()[0]
        print(f"Total apg_catalog records: {catalog_count}")
        
        # Check content records
        cur.execute("SELECT COUNT(*) FROM apg_content")
        content_count = cur.fetchone()[0]
        print(f"Total apg_content records: {content_count}")
        
        # Show summary of document sources
        cur.execute("SELECT document_source, COUNT(*) FROM apg_catalog GROUP BY document_source")
        sources = cur.fetchall()
        print("\nDocument sources in catalog:")
        for source, count in sources:
            print(f"  - {source}: {count} records")

get_database_overview(conn)

## Catalog Explorer

Explore the catalog table for a specific document source.

In [ ]:
def explore_catalog(conn, document_source="internal_wiki"):
    """Explore the catalog table for a specific document source."""
    if not conn:
        return
    
    with conn.cursor() as cur:
        # Get catalog entries for the specified source
        cur.execute("""
            SELECT id, document_type, document_name, document_description, 
                   date_created, date_last_modified, file_name, file_type, file_path
            FROM apg_catalog 
            WHERE document_source = %s
        """, (document_source,))
        
        records = cur.fetchall()
        if not records:
            print(f"No records found for document_source: {document_source}")
            return
        
        # Convert to DataFrame for better display
        columns = ["id", "document_type", "document_name", "document_description", 
                   "date_created", "date_last_modified", "file_name", "file_type", "file_path"]
        df = pd.DataFrame(records, columns=columns)
        
        # Print summary
        print(f"Found {len(records)} records for document_source: {document_source}")
        
        # Return DataFrame for further analysis
        return df

# Explore internal_wiki catalog (default)
catalog_df = explore_catalog(conn)
display(catalog_df)

## Content Explorer

Explore the content table for a specific document source or document name.

In [ ]:
def explore_content(conn, document_source="internal_wiki", document_name=None):
    """Explore the content table for a specific document source or document name."""
    if not conn:
        return
    
    with conn.cursor() as cur:
        if document_name:
            # Get content for a specific document
            cur.execute("""
                SELECT id, document_type, document_name, section_id, section_name, 
                       section_summary, LEFT(section_content, 100) as preview
                FROM apg_content 
                WHERE document_source = %s AND document_name = %s
                ORDER BY section_id
            """, (document_source, document_name))
            print(f"Content for document_source: {document_source}, document_name: {document_name}")
        else:
            # Get all content for the specified source
            cur.execute("""
                SELECT id, document_type, document_name, section_id, section_name, 
                       section_summary, LEFT(section_content, 100) as preview
                FROM apg_content 
                WHERE document_source = %s
                ORDER BY document_name, section_id
            """, (document_source,))
            print(f"Content for document_source: {document_source}")
        
        records = cur.fetchall()
        if not records:
            print("No content records found.")
            return
        
        # Convert to DataFrame for better display
        columns = ["id", "document_type", "document_name", "section_id", "section_name", 
                   "section_summary", "preview"]
        df = pd.DataFrame(records, columns=columns)
        
        # Print summary
        print(f"Found {len(records)} content records\n")
        
        # Return DataFrame for further analysis
        return df

# Explore all internal_wiki content
content_df = explore_content(conn)
display(content_df)

## Document ID Format Analysis

Analyze the document ID formats in the catalog and compare with what our code expects.

In [ ]:
def analyze_document_ids(catalog_df):
    """Analyze the document ID formats in the catalog."""
    if catalog_df is None or catalog_df.empty:
        print("No catalog data available for analysis.")
        return
    
    print("Document ID Analysis:")
    print(f"Total documents: {len(catalog_df)}")
    
    # Check if IDs are numeric
    numeric_ids = catalog_df['id'].apply(lambda x: str(x).isdigit())
    print(f"Documents with numeric IDs: {numeric_ids.sum()} ({numeric_ids.sum()/len(catalog_df)*100:.1f}%)")
    
    # Check for pattern like 'doc_123'
    prefix_ids = catalog_df['id'].apply(lambda x: str(x).startswith('doc_'))
    print(f"Documents with 'doc_' prefix: {prefix_ids.sum()} ({prefix_ids.sum()/len(catalog_df)*100:.1f}%)")
    
    # Display example IDs
    print("\nExample document IDs:")
    for idx, doc_id in enumerate(catalog_df['id'].head(5)):
        print(f"  {idx+1}. {doc_id} (type: {type(doc_id).__name__})")
    
    # Provide guidance
    if numeric_ids.all():
        print("\nAll document IDs are numeric. The internal_wiki subagent should be configured to use numeric IDs.")
    elif prefix_ids.all():
        print("\nAll document IDs have 'doc_' prefix. The internal_wiki subagent should be configured to expect this prefix.")
    else:
        print("\nMixed document ID formats detected. The internal_wiki subagent should handle multiple ID formats.")

# Analyze document IDs
analyze_document_ids(catalog_df)

## Get Full Document Content

Retrieve and display the full content of a specific document.

In [ ]:
def get_document_content(conn, document_id=None, document_name=None, document_source="internal_wiki"):
    """Get the full content of a document by ID or name."""
    if not conn:
        return
    
    if not document_id and not document_name:
        print("Please provide either document_id or document_name")
        return
    
    with conn.cursor() as cur:
        if document_id:
            # First get document name from ID
            cur.execute("""
                SELECT document_name FROM apg_catalog 
                WHERE id = %s AND document_source = %s
            """, (document_id, document_source))
            
            result = cur.fetchone()
            if not result:
                print(f"No document found with ID {document_id} in source {document_source}")
                return
                
            document_name = result[0]
            print(f"Found document name: {document_name} for ID: {document_id}")
        
        # Get document content
        cur.execute("""
            SELECT section_id, section_name, section_content 
            FROM apg_content 
            WHERE document_source = %s AND document_name = %s
            ORDER BY section_id
        """, (document_source, document_name))
        
        sections = cur.fetchall()
        if not sections:
            print(f"No content found for document {document_name} in source {document_source}")
            return
        
        print(f"\nDocument: {document_name} (source: {document_source})")
        print("=" * 80)
        
        for section_id, section_name, section_content in sections:
            section_title = section_name if section_name else f"Section {section_id}"
            print(f"\n## {section_title}")
            print("-" * 80)
            print(section_content)
            print("-" * 80)
        
        return sections

# Example: Get document by ID
# Replace the ID with an actual ID from your database
# get_document_content(conn, document_id=1)

## Test ID Formats

Test different ID formats to see which ones work with the database.

In [ ]:
def test_id_formats(conn, base_id=1):
    """Test different ID formats to see which ones work with the database."""
    if not conn:
        return
    
    # Test different ID formats
    formats = [
        base_id,                  # Integer
        str(base_id),             # String number
        f"doc_{base_id}",         # With doc_ prefix
        f"{base_id:04d}",         # Padded number
        f"document_{base_id}"      # With document_ prefix
    ]
    
    print(f"Testing different ID formats for base ID {base_id}...\n")
    
    for format_id in formats:
        print(f"Testing ID: {format_id} (type: {type(format_id).__name__})")
        
        with conn.cursor() as cur:
            try:
                # Test if this ID format works with straight equality
                cur.execute("""
                    SELECT document_name FROM apg_catalog 
                    WHERE id = %s
                """, (format_id,))
                
                result = cur.fetchone()
                if result:
                    print(f"  ✅ Direct match found: {result[0]}")
                else:
                    print(f"  ❌ No direct match found")
                    
                # Test if casting to text helps
                cur.execute("""
                    SELECT document_name FROM apg_catalog 
                    WHERE id::text = %s
                """, (format_id,))
                
                result = cur.fetchone()
                if result:
                    print(f"  ✅ Text cast match found: {result[0]}")
                else:
                    print(f"  ❌ No text cast match found")
                
                # Test with LIKE for prefix/suffix issues
                cur.execute("""
                    SELECT document_name FROM apg_catalog 
                    WHERE id::text LIKE %s
                """, (f"%{base_id}%",))
                
                result = cur.fetchone()
                if result:
                    print(f"  ✅ LIKE pattern match found: {result[0]}")
                else:
                    print(f"  ❌ No LIKE pattern match found")
                    
            except Exception as e:
                print(f"  ❌ Error: {str(e)}")
                
        print()

# Test different ID formats (replace with a valid ID in your database)
test_id_formats(conn, base_id=1)

## Query to Simulate Internal Wiki Subagent

This simulates the internal_wiki subagent's document fetch to troubleshoot ID issues.

In [ ]:
def simulate_subagent_fetch(conn, doc_ids=["1", "2", "3"]):
    """Simulate the internal_wiki subagent's document fetch."""
    if not conn:
        return
    
    print(f"Simulating subagent fetch with doc_ids: {doc_ids}\n")
    
    # Use document IDs directly without conversion
    if not doc_ids:
        print("No document IDs to fetch")
        return []
    
    result = []
    
    try:
        # First, get the document names for all requested IDs from the catalog
        doc_names = {}
        with conn.cursor() as cur:
            placeholders = ','.join(['%s'] * len(doc_ids))
            query = f"""
                SELECT id, document_name 
                FROM apg_catalog 
                WHERE id::text IN ({placeholders})
                AND document_source = 'internal_wiki'
            """
            print(f"Executing query: {query} with parameters: {doc_ids}")
            
            cur.execute(query, doc_ids)
            
            for row in cur.fetchall():
                doc_names[row[0]] = row[1]
                print(f"Found document: ID={row[0]}, Name={row[1]}")
        
        if not doc_names:
            print("No matching documents found in catalog")
            return []
        
        # Then, for each document, get its content sections
        for doc_id, doc_name in doc_names.items():
            with conn.cursor() as cur:
                cur.execute("""
                    SELECT section_id, section_name, section_content 
                    FROM apg_content 
                    WHERE document_source = 'internal_wiki'
                    AND document_name = %s
                    ORDER BY section_id
                """, (doc_name,))
                
                sections = []
                for row in cur.fetchall():
                    sections.append({
                        "section_name": row[1] if row[1] else f"Section {row[0]}",
                        "section_content": row[2]
                    })
                
                if sections:
                    print(f"Found {len(sections)} sections for document {doc_name}")
                    result.append({
                        "document_name": doc_name,
                        "sections": sections
                    })
                else:
                    print(f"No sections found for document {doc_name}")
        
        print(f"\nRetrieved content for {len(result)} documents")
        return result
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return []

# Test with document IDs
docs = simulate_subagent_fetch(conn, doc_ids=["1", "2", "3"])

# Also test with doc_ prefix
print("\n" + "-"*80 + "\n")
docs_with_prefix = simulate_subagent_fetch(conn, doc_ids=["doc_1", "doc_2", "doc_3"])

In [ ]:
# Close the connection when done
if conn:
    conn.close()
    print("Database connection closed.")